In [1]:
import sys
import os
import wandb
import torch
import numpy as np

from ml_collections import ConfigDict

# Add the parent directory to the sys.path
# project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), ""))
# sys.path.append(project_root)  # This allows importing from base, Elliptic, etc.
# sys.path.append(os.path.join(project_root, "Laplace_Exp")) 
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from Base.lla import dgala
from Base.utilities import clear_hooks
from files.training import train_dga,generate_data_elliptic,generate_data
from files.train_elliptic import train_elliptic


def get_burgers_config():
    config = ConfigDict()

    # Weights & Biases
    config.wandb = wandb = ConfigDict()
    wandb.project = "PDEs-training"
    wandb.name = "Burgers_MDNN"
    wandb.tag = None

    # General settings
    config.dga = "burgers"
    config.nn_model = "NN"  # Options: "NN", "WRF", "MDNN"
    config.lambdas = {"burgers":1,"uic":1, "ubcl":1, "ubcr":1}

    # Model-specific settings
    config.model = ConfigDict()
    config.model.input_dim = 3
    config.model.hidden_dim = 40
    config.model.num_layers = 3
    config.model.out_dim = 1
    config.model.activation = "tanh"

    # Training settings
    config.seed = 42
    config.learning_rate = 0.01
    config.decay_rate = 0.95
    config.weight_decay = 1e-3
    # config.alpha = 0.9  # For updating loss weights
    config.epochs = 1200
    config.start_scheduler = 1.0
    # config.weights_update = 250
    config.scheduler_step = 50

    config.samples = 150
    
    return config

def run_burgers_experiment(config_experiment,device):
    model_specific = f"_{config_experiment.nn_model}_s{config_experiment.samples}_wd{config_experiment.weight_decay}"
    model_path = f"./models/burgers" + model_specific + ".pth"
    model_dgala = f"./models/burgers_ll" + model_specific + ".pth"

    # Step 1: Training Phase (if needed)
    if config_experiment.train:
        print("Running Burgers model with " + model_specific)
        config = get_burgers_config()
        config.wandb.name = "burgers"+ model_specific
        config.weight_decay = config_experiment.weight_decay
        config.samples = config_experiment.samples  # Update the samples in the config
        pinn_nvs = train_dga(config, device=device)

    # Step 2: Fit deepGALA
    if config_experiment.deepgala:
        print("Starting DeepGaLA fitting for NN" + model_specific)
        nn_surrogate_model = torch.load(model_path)
        nn_surrogate_model.eval()

        data_int,ini_c, left_bc, right_bc = generate_data(size=config_experiment.samples, nparam = 1, seed = 10, burgers=True)

        data_trainig = {"data_fit": {"pde":data_int, "ic_loss":ini_c,"bc_loss":(left_bc, right_bc)}, 
                        "class_method": {"pde": ["burgers_pde"], "ic_loss":["u"], "bc_loss":["u","u"]},
                        "outputs": {"pde": ["burgers"], "ic_loss": ["uic"],"bc_loss":["ubcl","ubcr"]}}

        burgers_model = torch.load(model_path)
        burgers_model.eval()
        llp = dgala(burgers_model, prior_precision=config_experiment.weight_decay)

        llp.fit(data_trainig)
        # Before saving:
        clear_hooks(llp)
        torch.save(llp, model_dgala)


def get_heat_config():
    config = ConfigDict()

    # Weights & Biases
    config.wandb = wandb = ConfigDict()
    wandb.project = "PDEs-training"
    wandb.name = "Heat_MDNN"
    wandb.tag = None

    # General settings
    config.dga = "heat"
    config.nn_model = "NN"  # Options: "NN", "WRF", "MDNN"
    config.lambdas = {"heat":1,"uic":1, "ubcl":1, "ubcr":1}

    # Model-specific settings
    config.model = ConfigDict()
    config.model.input_dim = 3
    config.model.hidden_dim = 20
    config.model.num_layers = 2
    config.model.out_dim = 1
    config.model.activation = "tanh"

    # Training settings
    config.seed = 42
    config.learning_rate = 0.01
    config.decay_rate = 0.95
    config.weight_decay = 1e-3
    # config.alpha = 0.9  # For updating loss weights
    config.epochs = 5000
    config.start_scheduler = 1
    # config.weights_update = 250
    config.scheduler_step = 50

    config.samples = 150
    
    return config


def run_heat_experiment(config_experiment,device):
    model_specific = f"_{config_experiment.nn_model}_s{config_experiment.samples}_wd{config_experiment.weight_decay}"
    model_path = f"./models/heat" + model_specific + ".pth"
    model_dgala = f"./models/heat_ll" + model_specific + ".pth"

    # Step 1: Training Phase (if needed)
    if config_experiment.train:
        print("Running Heat model with " + model_specific)
        config = get_heat_config()
        config.wandb.name = "heat"+ model_specific
        config.weight_decay = config_experiment.weight_decay
        config.samples = config_experiment.samples  # Update the samples in the config
        pinn_nvs = train_dga(config, device=device)

    # Step 2: Fit deepGALA
    if config_experiment.deepgala:
        print("Starting DeepGaLA fitting for NN" + model_specific)
        nn_surrogate_model = torch.load(model_path)
        nn_surrogate_model.eval()

        data_int,ini_c, left_bc, right_bc = generate_data(size=config_experiment.samples, nparam = 1, seed = 10, burgers=False)

        data_trainig = {"data_fit": {"pde":data_int, "ic_loss":ini_c,"bc_loss":(left_bc, right_bc)}, 
                        "class_method": {"pde": ["heat_pde"], "ic_loss":["u"], "bc_loss":["u","u"]},
                        "outputs": {"pde": ["heat"], "ic_loss": ["uic"],"bc_loss":["ubcl","ubcr"]}}

        heat_model = torch.load(model_path)
        heat_model.eval()

        llp = dgala(heat_model, prior_precision=config_experiment.weight_decay)

        llp.fit(data_trainig)
        # Before saving:
        clear_hooks(llp)
        torch.save(llp, model_dgala)


def get_elliptic_config():
    config = ConfigDict()

    # Weights & Biases
    config.wandb = wandb = ConfigDict()
    wandb.project = "PDEs-training"
    wandb.name = "MDNN"
    wandb.tag = None

    # General settings
    config.nn_model = "MDNN"  # Options: "NN", "WRF", "MDNN"
    config.lambdas = {"elliptic":1, "ubcl":1, "ubcr":1}

    # Model-specific settings
    config.model = ConfigDict()
    config.model.input_dim = 3
    config.model.hidden_dim = 20
    config.model.num_layers = 2
    config.model.out_dim = 1
    config.model.activation = "tanh"

    # Config KL parameters
    config.nparameters = 2

    # Training settings
    config.seed = 42
    config.learning_rate = 0.001
    config.decay_rate = 0.95
    config.weight_decay = 1e-3
    # config.alpha = 0.9  # For updating loss weights
    config.epochs = 5000
    config.start_scheduler = 0.5
    # config.weights_update = 250
    config.scheduler_step = 50

    config.samples = 5000
    config.batch_size = 150

    
    return config


def run_elliptic_experiment(config_experiment,device):
    model_specific = f"_{config_experiment.nn_model}_s{config_experiment.samples}_wd{config_experiment.weight_decay}"
    model_path = f"./models/elliptic" + model_specific + ".pth"
    model_dgala = f"./models/elliptic_ll" + model_specific + ".pth"

    # Step 1: Training Phase (if needed)
    if config_experiment.train:
        print("Running Heat model with " + model_specific)
        config = get_elliptic_config()
        config.wandb.name = "elliptic" + model_specific
        config.weight_decay = config_experiment.weight_decay
        config.samples = config_experiment.samples  # Update the samples in the config
        pinn_nvs = train_elliptic(config, device=device)

    # Step 2: Fit deepGALA
    if config_experiment.deepgala:
        print("Starting DeepGaLA fitting for NN" + model_specific)
        nn_surrogate_model = torch.load(model_path)
        nn_surrogate_model.eval()

        data_int,left_bc,right_bc = generate_data_elliptic(size=config_experiment.samples) 

        data_trainig = {"data_fit": {"pde":data_int, "left_bc":left_bc,"right_bc":right_bc}, 
                    "class_method": {"pde": ["elliptic_pde"], "left_bc":["u"],"right_bc":["u"]},
                    "outputs": {"pde": ["elliptic"], "left_bc": ["ubcl"],"right_bc":["ubcr"]}}

        heat_model = torch.load(model_path)
        heat_model.eval()

        llp = dgala(heat_model, prior_precision=config_experiment.weight_decay)

        llp.fit(data_trainig)
        # Before saving:
        clear_hooks(llp)
        torch.save(llp, model_dgala)

def experiment():
    config = ConfigDict()
    
    # Train config
    config.train = True
    config.deepgala = True
    config.samples = 250
    config.nn_model = "NN"
    config.weight_decay = 0.

    return config


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Burgers & Heat Experiments
N = [500,1500]
nn_models = ["NN","MDNN"]
weight_decay = [0.,0.001]

config_experiment = experiment()

for model in nn_models:
    config_experiment.nn_model = model
    for nsamples in N:
        config_experiment.samples = nsamples
        for wd in weight_decay:
            config_experiment.weight_decay = wd
            run_burgers_experiment(config_experiment,device)
            run_heat_experiment(config_experiment,device)


N = [150,2500]

config_experiment = experiment()
for model in nn_models:
    config_experiment.nn_model = model
    for nsamples in N:
        config_experiment.samples = nsamples
        for wd in weight_decay:
            config_experiment.weight_decay = wd
            run_elliptic_experiment(config_experiment,device)

 

/home/s2113174/anaconda3/envs/fenicsx-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Running Burgers model with _NN_s500_wd0.0


wandb: Currently logged in as: christian-jb07 (christian-jb07-univeristy-of-edinburgh). Use `wandb login --relogin` to force relogin


Starting Training: LBFGS optimizer


epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_burgers,▁█▆▄▅▄▄▄▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▂▂▂▂▂▁▂▂▁
loss_computation_time,█▃▂▃▃▄▃▃▃▆▃▇▃▁▄▆▁▆▂▄▆█▇▃▂▇▅▅▆▇▇▄▃▅▂▄▃▂▄▁
loss_ubcl,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcr,█▆▇▂▂▁▂▁▁▁▁▁▂▁▂▁▁▁▁▁▂▁▁▂▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁
loss_uic,█▄▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▅▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,1199
learning_rate,0.01
loss_burgers,0.00573


...

Starting DeepGaLA fitting for NN_MDNN_s150_wd0.001
Running Heat model with _MDNN_s2500_wd0.0


epoch,▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█████
learning_rate,█████████████████▇▆▆▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁
loss_computation_time,▁▂▄▄▄▂▃▁▁▁▂▃▂▁▁▂▁▃▁▁▁▁▃▁▁▁█▁▂▂▃▂▁▁▁▂▂▁▁▂
loss_elliptic,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcl,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcr,█▇▆▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▆▆▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▅▅▄▃▂▂▂▂▂▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,8e-05
loss_computation_time,0.00114


Starting DeepGaLA fitting for NN_MDNN_s2500_wd0.0
Running Heat model with _MDNN_s2500_wd0.001


epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█
learning_rate,████████████████████▅▄▄▄▄▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁
loss_computation_time,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁
loss_elliptic,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcl,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcr,█▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▅▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,8e-05
loss_computation_time,0.00102


Starting DeepGaLA fitting for NN_MDNN_s2500_wd0.001


In [ ]:
import sys
import os
import wandb
import torch
import numpy as np

import pyro
import pyro.distributions as dist
import pyro.infer.mcmc as mcmc

from ml_collections import ConfigDict

# Add the parent directory to the sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from files.training import train_dga
from files.train_elliptic import train_elliptic,generate_data,samples_param


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Burgers

In [2]:

def get_burgers_config():
    config = ConfigDict()

    # Weights & Biases
    config.wandb = wandb = ConfigDict()
    wandb.project = "PDEs-training"
    wandb.name = "Burgers_MDNN"
    wandb.tag = None

    # General settings
    config.dga = "burgers"
    config.nn_model = "NN"  # Options: "NN", "WRF", "MDNN"
    config.lambdas = {"burgers":1,"uic":1, "ubcl":1, "ubcr":1}

    # Model-specific settings
    config.model = ConfigDict()
    config.model.input_dim = 3
    config.model.hidden_dim = 40
    config.model.num_layers = 3
    config.model.out_dim = 1
    config.model.activation = "tanh"

    # Weight-Random-Factorization
    #config.reparam = ConfigDict({"type":"weight_fact","mean":1.0,"stddev":0.1})

     # Periodic embeddings
    #config.model.period_emb = ConfigDict({"period":(1.0, 1.0), "axis":(0, 1) })

    # Fourier embeddings
    #config.model.fourier_emb = ConfigDict({"embed_scale":1,"embed_dim":256,"exclude_last_n":100})

    # Training settings
    config.seed = 42
    config.learning_rate = 0.01
    config.decay_rate = 0.95
    config.weight_decay = 1e-3
    # config.alpha = 0.9  # For updating loss weights
    config.epochs = 1200
    config.start_scheduler = 1.0
    # config.weights_update = 250
    config.scheduler_step = 50

    config.batch_size = 150
    
    return config

In [3]:
N = [500,1500]

#Loop through each sample size
for batch_size in N:
    print(f"Running training with {batch_size} samples...")
    config = get_burgers_config()
    config.wandb.name = f"Burgers_MDNN_s{batch_size}"
    config.batch_size = batch_size  # Update the samples in the config
    pinn_nvs = train_dga(config, device=device)
    print(f"Completed training with {batch_size} samples.")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Running training with 500 samples...


wandb: Currently logged in as: christian-jb07 (christian-jb07-univeristy-of-edinburgh). Use `wandb login --relogin` to force relogin


Starting Training: LBFGS optimizer


epoch,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_burgers,▁▁█▇▆▇▅█▄▃▃▅▃▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▃▂▁▂▂▂▁▂▂▁▂
loss_computation_time,█▄▂▃▄▇▂▄▅▅▃▃▁▂▇▂▅▄▄▂▃▅▄▂▄▁▄▅▃▃▂▄▄▄▇▃▄▆▂▅
loss_ubcl,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcr,▆█▄▆▇▃▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_uic,█▇▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,▆█▃▃▃▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,1199
learning_rate,0.01
loss_burgers,0.00459


Completed training with 500 samples.
Running training with 1500 samples...


Starting Training: LBFGS optimizer


epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇████
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_burgers,▁▁█▇▆█▆▅▅▄▅▄▄▄▄▃▃▃▂▃▄▃▃▃▂▂▂▂▃▄▃▂▂▂▂▂▂▂▂▂
loss_computation_time,▃▄▁▃▇▃▆▆▅█▇▆▆▄▆▂▅▇▅▅▅▆▄▆▇▇▆▇▅▄▆▄▄▄▆▅▆▅▅▅
loss_ubcl,█▆▅▅▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcr,▄▃▂█▇▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_uic,███▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,███▇▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,1199
learning_rate,0.01
loss_burgers,0.00709


Completed training with 1500 samples.


# Heat Eq

In [4]:

def get_heat_config():
    config = ConfigDict()

    # Weights & Biases
    config.wandb = wandb = ConfigDict()
    wandb.project = "PDEs-training"
    wandb.name = "Heat_MDNN"
    wandb.tag = None

    # General settings
    config.dga = "heat"
    config.nn_model = "NN"  # Options: "NN", "WRF", "MDNN"
    config.lambdas = {"heat":1,"uic":1, "ubcl":1, "ubcr":1}

    # Model-specific settings
    config.model = ConfigDict()
    config.model.input_dim = 3
    config.model.hidden_dim = 20
    config.model.num_layers = 2
    config.model.out_dim = 1
    config.model.activation = "tanh"

    # Weight-Random-Factorization
    #config.reparam = ConfigDict({"type":"weight_fact","mean":1.0,"stddev":0.1})

     # Periodic embeddings
    #config.model.period_emb = ConfigDict({"period":(1.0, 1.0), "axis":(0, 1) })

    # Fourier embeddings
    # config.model.fourier_emb = ConfigDict({"embed_scale":1,"embed_dim":20,"exclude_last_n":1})

    # Training settings
    config.seed = 42
    config.learning_rate = 0.01
    config.decay_rate = 0.95
    config.weight_decay = 1e-3
    # config.alpha = 0.9  # For updating loss weights
    config.epochs = 5000
    config.start_scheduler = 1
    # config.weights_update = 250
    config.scheduler_step = 50

    config.batch_size = 150
    
    return config

In [5]:
N = [500,1500]

#Loop through each sample size
for batch_size in N:
    print(f"Running training with {batch_size} samples...")
    config = get_heat_config()
    config.wandb.name = f"Heat_MDNN_s{batch_size}"
    config.batch_size = batch_size  # Update the samples in the config
    pinn_nvs = train_dga(config, device=device)
    print(f"Completed training with {batch_size} samples.")

Running training with 500 samples...


Starting Training: LBFGS optimizer


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_computation_time,▁▃▁▂▁▂▃▁█▂▁▁▁▁▂▂▁▂▁▁▁▁▃▂▁▂▁▁▁▁▁▁▂▁▁▂▂▁▁▂
loss_heat,▁▁▂▃▃▃▅▄▅▅▅▅▅▅▆▅▅▅▅▅▅▅▅▄▄▆█▆▆▇▅▆▅▅▆▅▅▅▅▅
loss_ubcl,▄█▇█▇██▅▅▃▂▅▃▅▁▃▅▂▃▄▂▄▂▁▂▃▁▁▃▁▄▂▂▂▂▃▃▁▃▂
loss_ubcr,▃▃▅▆▅▄▄▃▂▅▃▂▂▂█▄▂▃▁▂▂▃▅▁▁▂▂▂▂▁▂▃▂▁▅▄▂▃▂▇
loss_uic,███▆▆▅▅▅▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,███▇▅▄▄▄▃▃▂▃▂▃▂▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,0.01
loss_computation_time,0.00099


Completed training with 500 samples.
Running training with 1500 samples...


Starting Training: LBFGS optimizer


epoch,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_computation_time,▃▁▄▁▃▃▄▁▄▄▂▂▂▂▆▂▅▅▆▂█▂▅▃▂▂▃▅▃▄▂▄▃▅▅▄▃▁▂▁
loss_heat,▁▁▄▃▃▃▃▅▄▆▅▆▅▅▇▅▆▅▅▅▇▆▇▅▇▇▇▇█▇█▇▆▇▆▇▇▅▇▇
loss_ubcl,▁▇█▅▅▆▅▇▆▆▄▅▄▆▄▄▃▃▃▃▃▃▃▃▃▂▄▂▂▂▃▂▄▂▃▂▂▂▃▂
loss_ubcr,▁▁▆▄▇▄▄▄▃▄▂▃▃▃▅▅▁▄▁▃▂▅▄▄▄▁▂▃█▃▂▂▃▃▄▂▂▂▁▂
loss_uic,█▆▅▅▅▅▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train_loss,████▇▆▅▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,0.01
loss_computation_time,0.00344


Completed training with 1500 samples.


# Elliptic

In [6]:

def get_elliptic_config():
    config = ConfigDict()

    # Weights & Biases
    config.wandb = wandb = ConfigDict()
    wandb.project = "Elliptic-training"
    wandb.name = "MDNN"
    wandb.tag = None

    # General settings
    config.nn_model = "MDNN"  # Options: "NN", "WRF", "MDNN"
    config.lambdas = {"elliptic":1, "ubcl":1, "ubcr":1}

    # Model-specific settings
    config.model = ConfigDict()
    config.model.input_dim = 3
    config.model.hidden_dim = 20
    config.model.num_layers = 2
    config.model.out_dim = 1
    config.model.activation = "tanh"

    # Weight-Random-Factorization
    #config.reparam = ConfigDict({"type":"weight_fact","mean":1.0,"stddev":0.1})

     # Periodic embeddings
    #config.model.period_emb = ConfigDict({"period":(1.0, 1.0), "axis":(0, 1) })

    # Fourier embeddings
    #config.model.fourier_emb = ConfigDict({"embed_scale":1,"embed_dim":256,"exclude_last_n":100})

    # Config KL parameters
    config.nparameters = 2

    # Training settings
    config.seed = 42
    config.learning_rate = 0.001
    config.decay_rate = 0.95
    config.weight_decay = 1e-3
    # config.alpha = 0.9  # For updating loss weights
    config.epochs = 5000
    config.start_scheduler = 0.5
    # config.weights_update = 250
    config.scheduler_step = 50

    config.samples = 5000
    config.batch_size = 150

    
    return config

In [7]:
N = [150,2500]

#Loop through each sample size
for samples in N:
    print(f"Running training with {samples} samples...")
    config = get_elliptic_config()
    config.wandb.name = f"MDNN_s{samples}"
    config.samples = samples  # Update the samples in the config
    pinn_nvs = train_elliptic(config, device=device)
    print(f"Completed training with {samples} samples.")

Running training with 150 samples...


epoch,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇███
learning_rate,█████████████████████████▅▅▄▃▂▂▂▂▂▂▂▁▁▁▁
loss_computation_time,█▂▃▂▃▆▃▃▇▃▁▂▇▂▂▁▃▂▂▆▂▄▆▄█▃▃▃▃▅▄▄▄▅▇▂▂▅▃▃
loss_elliptic,█▆▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcl,█▅▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcr,█▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▅▄▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▅▄▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,8e-05
loss_computation_time,0.00298


Completed training with 150 samples.
Running training with 2500 samples...


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇██
learning_rate,████████████████████████▇▆▆▅▄▃▃▃▃▂▁▁▁▁▁▁
loss_computation_time,▁▁▂▂▁▂▃▃▂▅▁▁▂▁▁▁▁▂▁▁▁▁▁▁▅▁▁▁█▁▁▂▆▅▁▃▁▄▁▁
loss_elliptic,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcl,██▆▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcr,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▅▄▃▂▂▂▂▂▁▁▁▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,8e-05
loss_computation_time,0.0011


Completed training with 2500 samples.
